In [1]:
import Data.Manifold
import Data.Manifold.Web

import Data.Manifold.TreeCover
import Data.Manifold.Riemannian
import Linear.V2
import Data.Foldable (toList)
import Control.Arrow
import Data.Semigroup (Option(..))
import Data.Maybe (mapMaybe)
import Control.Lens

From [dynamic-plot](http://hackage.haskell.org/packages/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2

In [3]:
import Data.Colour
import Data.Colour.Names

In [4]:
(...) :: Colour ℝ -> Colour ℝ -> Shade (Colour ℝ)
c₀...c₁ = case rangeOnGeodesic c₀ c₁ of
    Option (Just interp) -> interp (0 :±[1] :: Shade ℝ)

In [5]:
toInterior =<< ($ 0) <$> (interpolate blue yellow :: Option (ℝ -> Colour ℝ))

Option {getOption = Just (ColourNeedle {getRGBNeedle = RGB {channelRed = 0.0, channelGreen = 0.0, channelBlue = 0.0}})}

In [6]:
let Option (Just interp) = interpolate blue yellow :: Option (ℝ->Colour ℝ)
    Option (Just c₀) = toInterior (interp 1e-17)
    Option (Just c₁) = toInterior (interp $ -1)
    Option (Just c₂) = toInterior (interp 1)
  in [fromInterior c₀, fromInterior c₁, fromInterior c₂ :: Colour ℝ]

[Data.Colour.SRGB.Linear.rgb 0.5 0.5 0.5,Data.Colour.SRGB.Linear.rgb 0.11920292202211757 0.11920292202211757 0.8807970779778824,Data.Colour.SRGB.Linear.rgb 0.8807970779778824 0.8807970779778824 0.11920292202211757]

In [7]:
fromInterior $ (head . pointsShades $ mapMaybe (getOption . toInterior)
   [turquoise, beige :: Colour ℝ]
     :: Shade (Colour ℝ))^.shadeCtr :: Colour ℝ
[turquoise, fromInterior ((turquoise...turquoise)^.shadeCtr) :: Colour ℝ]

Data.Colour.SRGB.Linear.rgb 0.19031803090275262 0.8692737099119894 0.6772205205892864

[Data.Colour.SRGB.Linear.rgb 5.126945837404324e-2 0.7454042095403874 0.6307571363461468,Data.Colour.SRGB.Linear.rgb 5.126945837404334e-2 0.7454042095403874 0.6307571363461468]

In [8]:
iWeb :: PointsWeb ℝ² (Colour ℝ)
iWeb = fromWebNodes euclideanMetric
   [     (V2 0.5 0, blue),    (V2 1.5 0, beige),   (V2 2.5 0, teal)
   , (V2 0 1, red),  (V2 1 1, violet),(V2 2 1, green),  (V2 3 1, turquoise)
   , (V2 0 2, gray), (V2 1 2, indigo),(V2 2 2, crimson),(V2 3 2, orange)
   , (V2 0 3, brown),(V2 1 3, black), (V2 2 3, cyan),   (V2 3 3, royalblue) ]
plotWindow [plot iWeb, dynamicAxes]

GraphWindowSpecR2{lBound=-0.5, rBound=3.5, bBound=-0.5, tBound=3.5, xResolution=640, yResolution=480}

![Simple “Guroud shaded” web of some colours](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/WebSampling/simpleColourWeb.png)

In [9]:
uWeb :: PointsWeb ℝ² (Shade (Colour ℝ))
uWeb = fromWebNodes euclideanMetric
   [ (V2 0 0, blue...yellow),(V2 1 0, beige...red),   (V2 2 0, grey...teal)
   , (V2 0 1, red...grey),   (V2 1 1, green...violet),(V2 2 1, orange...cyan)
   , (V2 0 2, gray...green), (V2 1 2, blue...gold),   (V2 2 2, red...lightgreen) ]
plotWindow [plot uWeb, dynamicAxes]

GraphWindowSpecR2{lBound=-0.33333333333333326, rBound=2.333333333333334, bBound=-0.33333333333333326, tBound=2.333333333333334, xResolution=640, yResolution=480}

![Web of “uncertain colours”, rendered as random pertubations (“noise”)](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/WebSampling/noisyColourWeb.png)